# Plant Leaf Classification

data:

<https://data.mendeley.com/datasets/tywbtsjrjv/1>

<https://drive.google.com/drive/folders/1QswvBejKJrc9tz7nNsxGxJblR2vwo6na?usp=sharing>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
os.chdir("drive/")
os.chdir('My Drive')
os.chdir('Tutorials')
os.chdir('DeepLearningProject')
os.chdir('LeafDiseaseDetection')

## Data Preparation

In [3]:
import os
import shutil

original_dataset_dir = './dataset'
classes_list = os.listdir(original_dataset_dir)

base_dir = './splitted'
ALREADY_SPLITTED = True
if not os.path.exists(base_dir):
    ALREADY_SPLITTED = False
    os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')

if not os.path.exists(train_dir):
    os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)


if not ALREADY_SPLITTED:
    for cls in classes_list:
        os.mkdir(os.path.join(train_dir, cls))
        os.mkdir(os.path.join(validation_dir, cls))
        os.mkdir(os.path.join(test_dir, cls))


In [4]:
import math


for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)

    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)

    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)

    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)


Train size( Strawberry___healthy ):  600
Validation size( Strawberry___healthy ):  200
Test size( Strawberry___healthy ):  200
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Potato___Early_blight ):  600
Validation size( Potato___Early_blight ):  200
Test size( Potato___Early_blight ):  200
Train size( Blueberry___healthy ):  901
Validation size( Blueberry___healthy ):  300
Test size( Blueberry___healthy ):  300
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210
Train size( Tomato___Target_Spot ):  842
Validation size( Tomato___Target_Spot ):  280
Test size( Tomato___Target_Spot ):  280
Train size( Peach___healthy ):  600
Validation size( Peach___healthy ):  200
Test size( Peach___healthy ):  200
Train size( Potato___Late_blight ):  600
Validation size( Potato___Late_blight ):  200
Test size( Potato___Late_blight ):  200

## Training

In [6]:
import torch
import os

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
if torch.backends.mps.is_available():
    # torch.mps.set_start_method('forkserver')
    DEVICE = torch.device("mps")

BATCH_SIZE = 256
EPOCH = 30

In [7]:
print("CUDA Available: ",USE_CUDA)
print("Device: ",DEVICE)

CUDA Available:  False
Device:  mps


In [8]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder


transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [9]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)

        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 39)

    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)

        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)


model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [15]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [16]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output,target, reduction='sum').item()


            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [17]:
import time
import copy


def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    model.load_state_dict(best_model_wts)
    return model



base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)
torch.save(base,'baseline.pt')

-------------- epoch 1 ----------------
train Loss: 1.3961, Accuracy: 58.68%
val Loss: 1.4116, Accuracy: 58.01%
Completed in 1m 30s
-------------- epoch 2 ----------------
train Loss: 0.8478, Accuracy: 74.44%
val Loss: 0.8872, Accuracy: 73.43%
Completed in 1m 24s
-------------- epoch 3 ----------------
train Loss: 0.6518, Accuracy: 80.85%
val Loss: 0.6932, Accuracy: 79.27%
Completed in 1m 24s
-------------- epoch 4 ----------------
train Loss: 0.5244, Accuracy: 83.21%
val Loss: 0.5876, Accuracy: 81.26%
Completed in 1m 24s
-------------- epoch 5 ----------------
train Loss: 0.4623, Accuracy: 85.44%
val Loss: 0.5306, Accuracy: 83.38%
Completed in 1m 26s
-------------- epoch 6 ----------------
train Loss: 1.1426, Accuracy: 67.00%
val Loss: 1.2404, Accuracy: 64.76%
Completed in 1m 28s
-------------- epoch 7 ----------------
train Loss: 0.3366, Accuracy: 89.78%
val Loss: 0.4235, Accuracy: 86.69%
Completed in 1m 28s
-------------- epoch 8 ----------------
train Loss: 0.3423, Accuracy: 89.11%

## Transfer Learning

In [18]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),

    'val': transforms.Compose([transforms.Resize([64,64]),
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

In [19]:
data_dir = './splitted'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

* Load Pre-Trained Model

In [21]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [22]:
from torchvision import models
from torch.optim import lr_scheduler


resnet = models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 33)
resnet = resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)


exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/ywsung/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%


* Freeze first few layers of the pre-trained model

In [23]:
ct = 0
for child in resnet.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

In [26]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch+1))
        since = time.time()
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0


            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects/dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))


            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)

    return model

In [27]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)

torch.save(model_resnet50, 'resnet50.pt')

-------------- epoch 1 ----------------
train Loss: 0.1702 Acc: 0.7396
val Loss: 0.1334 Acc: 0.7484
Completed in 1m 45s
-------------- epoch 2 ----------------
train Loss: 0.1043 Acc: 0.7576
val Loss: 0.1154 Acc: 0.7554
Completed in 1m 44s
-------------- epoch 3 ----------------
train Loss: 0.1101 Acc: 0.7576
val Loss: 0.0975 Acc: 0.7617
Completed in 1m 44s
-------------- epoch 4 ----------------
train Loss: 0.0976 Acc: 0.7614
val Loss: 0.0918 Acc: 0.7637
Completed in 1m 44s
-------------- epoch 5 ----------------
train Loss: 0.0765 Acc: 0.7682
val Loss: 0.0751 Acc: 0.7687
Completed in 1m 44s
-------------- epoch 6 ----------------
train Loss: 0.0570 Acc: 0.7737
val Loss: 0.0987 Acc: 0.7629
Completed in 1m 44s
-------------- epoch 7 ----------------
train Loss: 0.0388 Acc: 0.7792
val Loss: 0.0292 Acc: 0.7825
Completed in 1m 43s
-------------- epoch 8 ----------------
train Loss: 0.0188 Acc: 0.7863
val Loss: 0.0249 Acc: 0.7835
Completed in 1m 44s
-------------- epoch 9 ----------------


## Evaluation

In [28]:
transform_base = transforms.Compose([transforms.Resize([64,64]),transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test',transform=transform_base)

test_loader_base = torch.utils.data.DataLoader(
    test_base,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

In [29]:
transform_resNet = transforms.Compose([
    transforms.Resize([64,64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet)
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [30]:
baseline=torch.load('baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test acc:  ', test_accuracy)

baseline test acc:   94.37708519814468


In [31]:
resnet50=torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('ResNet test acc:  ', test_accuracy)

ResNet test acc:   78.56619741231997
